In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Define number of records
num_records = 2000

# Generate synthetic data
data = {
    'Cell Tower': [f'Tower {i+1}' for i in range(num_records)],
    'Latitude': np.random.uniform(39.5, 40.0, size=num_records),
    'Longitude': np.random.uniform(-76.0, -75.5, size=num_records),
    'Infrastructure_Capacity_Mbps': np.random.randint(80, 150, size=num_records),
    'Signal_Strength_dBm': np.random.randint(-90, -70, size=num_records),
    'Coverage_Area_sqkm': np.random.uniform(8.0, 15.0, size=num_records),
    'Latency_ms': np.random.randint(15, 25, size=num_records),
    'Packet_Loss_Percentage': np.random.uniform(0.2, 1.0, size=num_records),
    'Max_Connections': np.random.randint(800, 1500, size=num_records),
    'Existing_Customers': np.random.randint(1000, 2000, size=num_records),
    'Revenue_Loss_Per_Hour_USD': np.random.randint(800, 1200, size=num_records),
    'Overlap_Towers': [np.random.choice([f'Tower {j+1}' for j in range(num_records)], np.random.randint(1, 5), replace=False).tolist() for _ in range(num_records)],
    'SLA_Hours': np.random.randint(20, 30, size=num_records),
    'Operational_Status': np.random.choice(['Operational', 'Maintenance', 'Outage'], size=num_records),
    'Backup_Available': np.random.choice(['No Backup', 'Partial Backup', 'Full Backup'], size=num_records),
    'Connections_Handled': np.random.randint(900, 1400, size=num_records),
    'Backup_Available_Quantitative': np.random.randint(0, 3, size=num_records),  # Mapping No Backup: 0, Partial Backup: 1, Full Backup: 2
    'Outage_Type': np.random.choice(['Man-made', 'Natural Disaster', 'Infrastructure Deficit'], size=num_records),
    'Outage_Cause': np.random.choice(['Vandalism', 'Equipment Failure', 'Human Error', 'Storm', 'Earthquake', 'Flood', 'Power Outage', 'Network Equipment Failure', 'Backhaul Link Failure'], size=num_records),
    'Affected_Towers': [np.random.choice([f'Tower {j+1}' for j in range(num_records)], np.random.randint(1, 5), replace=False).tolist() for _ in range(num_records)],
    'Start_Date': [datetime.now() - timedelta(days=np.random.randint(1, 30)) for _ in range(num_records)],
    'End_Date': [datetime.now() - timedelta(days=np.random.randint(1, 30)) + timedelta(hours=np.random.randint(1, 24)) for _ in range(num_records)]
}

# Create DataFrame
df = pd.DataFrame(data)

In [2]:
# Function to calculate derived attributes based on overlap
def calculate_derived_attributes(row, attribute):
    if len(row['Overlap_Towers']) > 0:
        if attribute == 'Combined_Signal_Strength_dBm':
            return df.loc[df['Cell Tower'].isin(row['Overlap_Towers']), 'Signal_Strength_dBm'].mean()
        elif attribute == 'Combined_Coverage_Area_sqkm':
            return df.loc[df['Cell Tower'].isin(row['Overlap_Towers']), 'Coverage_Area_sqkm'].sum()
        elif attribute == 'Combined_Packet_Loss_Percentage':
            return df.loc[df['Cell Tower'].isin(row['Overlap_Towers']), 'Packet_Loss_Percentage'].mean()
        elif attribute == 'Combined_Max_Connections':
            return df.loc[df['Cell Tower'].isin(row['Overlap_Towers']), 'Max_Connections'].max()
    else:
        return np.nan
    
    
df['Num_Overlap_Towers'] = df['Overlap_Towers'].apply(len)

# Extent of Outage
df['Outage_Duration_Hours'] = (df['End_Date'] - df['Start_Date']).dt.total_seconds() / 3600
# Extract day of the week (Monday = 0, Sunday = 6)
df['Day_of_Week'] = df['Start_Date'].dt.dayofweek

# Extract hour of the day
df['Hour_of_Day'] = df['Start_Date'].dt.hour

# Extract month and season
df['Month'] = df['Start_Date'].dt.month
df['Season'] = df['Start_Date'].dt.month % 12 // 3 + 1  # 1: Winter, 2: Spring, 3: Summer, 4: Fall

# Categorize work hours vs. non-work hours
df['Is_Work_Hours'] = df['Start_Date'].dt.hour.between(9, 17)


# Calculate derived attributes
df['Combined_Signal_Strength_dBm'] = df.apply(lambda row: calculate_derived_attributes(row, 'Combined_Signal_Strength_dBm'), axis=1)
df['Combined_Coverage_Area_sqkm'] = df.apply(lambda row: calculate_derived_attributes(row, 'Combined_Coverage_Area_sqkm'), axis=1)
df['Combined_Packet_Loss_Percentage'] = df.apply(lambda row: calculate_derived_attributes(row, 'Combined_Packet_Loss_Percentage'), axis=1)
df['Combined_Max_Connections'] = df.apply(lambda row: calculate_derived_attributes(row, 'Combined_Max_Connections'), axis=1)

# Generate Impact Level based on some logic (example: random assignment)
df['Impact_Level'] = np.random.choice(['Catastrophic', 'Critical', 'Major', 'Minor', 'Information'], size=num_records)


In [3]:
df.head()

Cell Tower   Latitude  Longitude  Infrastructure_Capacity_Mbps  \
0    Tower 1  39.960424 -75.604494                           123   
1    Tower 2  39.844953 -75.721242                           125   
2    Tower 3  39.807379 -75.686078                            82   
3    Tower 4  39.990024 -75.605272                            80   
4    Tower 5  39.609642 -75.753683                           120   

   Signal_Strength_dBm  Coverage_Area_sqkm  Latency_ms  \
0                  -78            8.798861          22   
1                  -77           14.549703          21   
2                  -76            9.873776          19   
3                  -81           10.871349          18   
4                  -83           13.978957          17   

   Packet_Loss_Percentage  Max_Connections  Existing_Customers  ...  \
0                0.371237              877                1299  ...   
1                0.473760             1272                1481  ...   
2                0.257533              809                1317  ...   
3                0.359577             1359                1609  ...   
4                0.996695             1213                1419  ...   

   Day_of_Week Hour_of_Day  Month Season Is_Work_Hours  \
0            2          19      5      2         False   
1            2          19      5      2         False   
2            6          19      5      2         False   
3            5          19      6      3         False   
4            2          19      5      2         False   

   Combined_Signal_Strength_dBm  Combined_Coverage_Area_sqkm  \
0                    -75.000000                    20.591794   
1                    -86.333333                    42.064257   
2                    -80.750000                    44.323729   
3                    -79.000000                    11.737687   
4                    -89.000000                    14.170428   

  Combined_Packet_Loss_Percentage Combined_Max_Connections  Impact_Level  
0                        0.522269                     1479  Catastrophic  
1                        0.839317                     1435   Information  
2                        0.811097                     1248  Catastrophic  
3                        0.698696                     1091         Minor  
4                        0.841880                     1342   Information  

[5 rows x 34 columns]

In [4]:
df.columns

Index(['Cell Tower', 'Latitude', 'Longitude', 'Infrastructure_Capacity_Mbps',
       'Signal_Strength_dBm', 'Coverage_Area_sqkm', 'Latency_ms',
       'Packet_Loss_Percentage', 'Max_Connections', 'Existing_Customers',
       'Revenue_Loss_Per_Hour_USD', 'Overlap_Towers', 'SLA_Hours',
       'Operational_Status', 'Backup_Available', 'Connections_Handled',
       'Backup_Available_Quantitative', 'Outage_Type', 'Outage_Cause',
       'Affected_Towers', 'Start_Date', 'End_Date', 'Num_Overlap_Towers',
       'Outage_Duration_Hours', 'Day_of_Week', 'Hour_of_Day', 'Month',
       'Season', 'Is_Work_Hours', 'Combined_Signal_Strength_dBm',
       'Combined_Coverage_Area_sqkm', 'Combined_Packet_Loss_Percentage',
       'Combined_Max_Connections', 'Impact_Level'],
      dtype='object')

In [5]:
# Save dataset to CSV (optional)
df.to_csv('synthetic_dataset_with_derived.csv', index=False)

In [7]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = ['Operational_Status', 'Backup_Available', 'Outage_Type', 'Outage_Cause']
label_encoders = {}
# Apply label encoding to 'Operational_Status'
for col in categorical_cols:
    if col in df.columns:
        label_encoder = LabelEncoder()
        df[col + '_Encoded'] = label_encoder.fit_transform(df[col])
        label_encoders[col] = label_encoder
        
df.drop(categorical_cols, axis=1, inplace=True)

In [8]:
df.columns

Index(['Cell Tower', 'Latitude', 'Longitude', 'Infrastructure_Capacity_Mbps',
       'Signal_Strength_dBm', 'Coverage_Area_sqkm', 'Latency_ms',
       'Packet_Loss_Percentage', 'Max_Connections', 'Existing_Customers',
       'Revenue_Loss_Per_Hour_USD', 'Overlap_Towers', 'SLA_Hours',
       'Connections_Handled', 'Backup_Available_Quantitative',
       'Affected_Towers', 'Start_Date', 'End_Date', 'Num_Overlap_Towers',
       'Outage_Duration_Hours', 'Day_of_Week', 'Hour_of_Day', 'Month',
       'Season', 'Is_Work_Hours', 'Combined_Signal_Strength_dBm',
       'Combined_Coverage_Area_sqkm', 'Combined_Packet_Loss_Percentage',
       'Combined_Max_Connections', 'Impact_Level',
       'Operational_Status_Encoded', 'Backup_Available_Encoded',
       'Outage_Type_Encoded', 'Outage_Cause_Encoded'],
      dtype='object')

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming df_full_dataset contains your complete dataset with all attributes including 'Impact_Level'

# Define features and target variable
features = [ 'Infrastructure_Capacity_Mbps',
       'Signal_Strength_dBm', 'Coverage_Area_sqkm', 'Latency_ms',
       'Packet_Loss_Percentage', 'Max_Connections', 'Existing_Customers',
       'Revenue_Loss_Per_Hour_USD', 'SLA_Hours',
       'Connections_Handled', 'Backup_Available_Quantitative',
        'Num_Overlap_Towers','Day_of_Week', 'Hour_of_Day', 'Month', 'Season', 'Is_Work_Hours',
       'Combined_Signal_Strength_dBm', 'Combined_Coverage_Area_sqkm',
       'Combined_Packet_Loss_Percentage', 'Combined_Max_Connections',
      'Outage_Duration_Hours', 'Operational_Status_Encoded',
       'Backup_Available_Encoded', 'Outage_Type_Encoded',
       'Outage_Cause_Encoded']

target = 'Impact_Level'

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 968 to 1126
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Infrastructure_Capacity_Mbps     1600 non-null   int32  
 1   Signal_Strength_dBm              1600 non-null   int32  
 2   Coverage_Area_sqkm               1600 non-null   float64
 3   Latency_ms                       1600 non-null   int32  
 4   Packet_Loss_Percentage           1600 non-null   float64
 5   Max_Connections                  1600 non-null   int32  
 6   Existing_Customers               1600 non-null   int32  
 7   Revenue_Loss_Per_Hour_USD        1600 non-null   int32  
 8   SLA_Hours                        1600 non-null   int32  
 9   Connections_Handled              1600 non-null   int32  
 10  Backup_Available_Quantitative    1600 non-null   int32  
 11  Num_Overlap_Towers               1600 non-null   int64  
 12  Day_of_Week       

In [11]:
# Initialize RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Predict on test set
y_pred = clf.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.17
              precision    recall  f1-score   support

Catastrophic       0.14      0.24      0.18        62
    Critical       0.29      0.17      0.21        89
 Information       0.19      0.23      0.21        96
       Major       0.12      0.12      0.12        78
       Minor       0.17      0.11      0.13        75

    accuracy                           0.17       400
   macro avg       0.18      0.17      0.17       400
weighted avg       0.19      0.17      0.17       400



In [12]:
clf.feature_importances_

array([0.0565392 , 0.0458437 , 0.06150755, 0.03694577, 0.06110387,
       0.05889635, 0.05849351, 0.05937303, 0.03882918, 0.0595884 ,
       0.01780573, 0.01659359, 0.0314348 , 0.        , 0.00792834,
       0.00800094, 0.        , 0.05331955, 0.05730111, 0.06093865,
       0.05779822, 0.05758843, 0.01807559, 0.0197731 , 0.01872445,
       0.03759695])

In [15]:
import numpy as np 
feature_importances = clf.feature_importances_
norm_importance = feature_importances / np.sum(feature_importances)

In [18]:
clf.feature_importances_.sum()

1.0

In [20]:
probabilities = np.array([
    [0.1, 0.2, 0.3, 0.2, 0.2],  # Instance 1
    [0.3, 0.1, 0.2, 0.1, 0.3]   # Instance 2
])

In [21]:
priority_scores = np.zeros((probabilities.shape[0], probabilities.shape[1])) 
priority_scores

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [22]:
priority_scores.shape

(2, 5)

In [ ]:
priority_scores = np.zeros((probabilities.shape[0], probabilities.shape[1]))  # Initialize priority scores

for instance_idx in range(probabilities.shape[0]):
    for class_idx in range(probabilities.shape[1]):
        score_for_class = np.dot(norm_importance, probabilities[instance_idx])
        priority_scores[instance_idx, class_idx] = score_for_class * 100

In [ ]:
# # Compute combined benefits considering overlaps
# df_towers['Combined_Signal_Strength_dBm'] = df_towers.apply(
#     lambda row: (row['Signal_Strength_dBm'] +
#                  sum(df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Signal_Strength_dBm'])) / (len(row['Overlap_Towers']) + 1),
#     axis=1
# )

# df_towers['Combined_Coverage_Area_sqkm'] = df_towers.apply(
#     lambda row: (row['Coverage_Area_sqkm'] +
#                  sum(df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Coverage_Area_sqkm'])) / (len(row['Overlap_Towers']) + 1),
#     axis=1
# )

# df_towers['Combined_Packet_Loss_Percentage'] = df_towers.apply(
#     lambda row: (row['Packet_Loss_Percentage'] +
#                  sum(df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Packet_Loss_Percentage'])) / (len(row['Overlap_Towers']) + 1),
#     axis=1
# )

# df_towers['Combined_Max_Connections'] = df_towers.apply(
#     lambda row: (row['Max_Connections'] +
#                  sum(df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Max_Connections'])) / (len(row['Overlap_Towers']) + 1),
#     axis=1
# )


# df_towers